# Opdracht SVM & Hyperparametertuning

Support Vector Machines behoren nog altijd tot een van de meest populaire en gebruikte ML algoritmes. Ze zijn bijzonder veelzijdig en kunnen ingezet worden bij zowel classificatieproblemen als regressieproblemen en zelfs bij het opsporen van uitschieters.

Bij deze opdracht zullen we ons beperken tot SVM's voor classificatie. 

In een eerste deel hernemen we de cancer detection uit de vorige opdracht (logistic regression) waarbij we nu gebruik maken van een Support Vector Machine classifier.

Het tweede deel van de opdracht bestaat erin om handschriftherkenner te bouwen. We gebruiken daarvoor de MNIST dataset die bestaat uit duizenden afbeeldingen van handgeschreven cijfers.                                                                                          

In [75]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from skopt import BayesSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
import matplotlib.image as mpimg
from skimage.io import imread, imshow

# scaling
from sklearn import preprocessing

# lineare model
from sklearn import linear_model

# polynomial features
from sklearn.preprocessing import PolynomialFeatures

# svm
from sklearn import svm

# poly 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

## Opdracht 1 - Cancer detection via SVM

Doe de cancer detection uit vorige opdracht opnieuw maar nu gebruik makende van Support Vector Machines. 
Beantwoord ook telkens de vragen.

### Cancer detection

In [76]:
# Inlezen van de dataset

dataset = pd.read_csv('cancer.csv')
dataset.head(100)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.086900,0.070170,...,23.41,158.80,1956.0,0.1238,0.18660,0.24160,0.18600,0.2750,0.08902,NaN
1,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.197400,0.127900,...,25.53,152.50,1709.0,0.1444,0.42450,0.45040,0.24300,0.3613,0.08758,NaN
2,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.105200,...,26.50,98.87,567.7,0.2098,0.86630,0.68690,0.25750,0.6638,0.17300,NaN
3,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,...,16.67,152.20,1575.0,0.1374,0.20500,0.40000,0.16250,0.2364,0.07678,NaN
4,843786,M,12.450,15.70,82.57,477.1,0.12780,0.17000,0.157800,0.080890,...,23.75,103.40,741.6,0.1791,0.52490,0.53550,0.17410,0.3985,0.12440,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,86211,B,12.180,17.84,77.79,451.1,0.10450,0.07057,0.024900,0.029410,...,20.92,82.14,495.2,0.1140,0.09358,0.04980,0.05882,0.2227,0.07376,NaN
96,862261,B,9.787,19.94,62.11,294.5,0.10240,0.05301,0.006829,0.007937,...,26.29,68.81,366.1,0.1316,0.09473,0.02049,0.02381,0.1934,0.08988,NaN
97,862485,B,11.600,12.84,74.34,412.6,0.08983,0.07525,0.041960,0.033500,...,17.16,82.96,512.5,0.1431,0.18510,0.19220,0.08449,0.2772,0.08756,NaN
98,862548,M,14.420,19.77,94.48,642.5,0.09752,0.11410,0.093880,0.058390,...,30.86,109.50,826.4,0.1431,0.30260,0.31940,0.15650,0.2718,0.09353,NaN


## Data exploration

In [77]:
# gebalanceerdheid controleren
positive = dataset.loc[dataset['diagnosis'] == 'M'].count()
negative = dataset.loc[dataset['diagnosis'] == 'B'].count()
print(positive)
print(negative)

# data is unbalanced

id                         211
diagnosis                  211
radius_mean                211
texture_mean               211
perimeter_mean             211
area_mean                  211
smoothness_mean            211
compactness_mean           211
concavity_mean             211
concave points_mean        211
symmetry_mean              211
fractal_dimension_mean     211
radius_se                  211
texture_se                 211
perimeter_se               211
area_se                    211
smoothness_se              211
compactness_se             211
concavity_se               211
concave points_se          211
symmetry_se                211
fractal_dimension_se       211
radius_worst               211
texture_worst              211
perimeter_worst            211
area_worst                 211
smoothness_worst           211
compactness_worst          211
concavity_worst            211
concave points_worst       211
symmetry_worst             211
fractal_dimension_worst    211
Unnamed:

In [78]:
dataset.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.680000e+02,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,...,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,0.0
mean,3.042382e+07,14.120491,19.305335,91.914754,654.279754,0.096321,0.104036,0.088427,0.048746,0.181055,...,25.691919,107.125053,878.578873,0.132316,0.253541,0.271414,0.114341,0.289776,0.083884,NaN
std,1.251246e+08,3.523416,4.288506,24.285848,351.923751,0.014046,0.052355,0.079294,0.038617,0.027319,...,6.141662,33.474687,567.846267,0.022818,0.156523,0.207989,0.065484,0.061508,0.018017,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692225e+05,11.697500,16.177500,75.135000,420.175000,0.086290,0.064815,0.029540,0.020310,0.161900,...,21.095000,84.102500,514.975000,0.116600,0.146900,0.114475,0.064730,0.250350,0.071412,NaN
50%,9.061570e+05,13.355000,18.855000,86.210000,548.750000,0.095865,0.092525,0.061400,0.033455,0.179200,...,25.425000,97.655000,685.550000,0.131300,0.211850,0.226550,0.099840,0.282050,0.080015,NaN
75%,8.825022e+06,15.780000,21.802500,103.875000,782.625000,0.105300,0.130400,0.129650,0.073730,0.195625,...,29.757500,125.175000,1073.500000,0.146000,0.337600,0.381400,0.161325,0.317675,0.092065,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


## Data preprocessing

In [79]:
# Verwijder de laatste kolom en de kolom id

dataset.drop('id', axis=1, inplace=True)
dataset.drop('Unnamed: 32', axis=1, inplace=True)

# Vervangen van B en M door 0 en 1 in de output diagnosis
dataset['diagnosis'] = dataset['diagnosis'].map({'M': 1, 'B': 0})

# Opsplitsen in features en targets
X = dataset.drop(['diagnosis'], axis=1).values
y = dataset['diagnosis'].values
print(X.shape)
# Opsplitsen in training set en test set. Zorg dat de test set 1/3 is van de totale dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print('Shape:',X_train.shape)
print(X_test.shape)
# Normalisatie van de features
scaler = preprocessing.StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


(568, 30)
Shape: (397, 30)
(171, 30)


Train een logistic regression model via cross-validation.

- Gebruik daarvoor zowel grid search, random search en Bayes optimization om op zoek te gaan naar de beste hyperparameters: C-waarde, class_weight, solver. Meer info: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- Varieer bij K-fold cross-validation de waarde van K. Bespreek de resultaten.
- Test de bekomen modellen op de test set. Welke search techniek geniet jouw voorkeur en waarom?
- Heeft het zin om de featureset uit te breiden met polynomial features? Test dit. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

## Initialiseren en trainen van de logistic regression classifier

In [80]:
# Trainen van logistic regression via cross-validation

# Initialiseren van de logistic regression classifier

logreg = linear_model.LogisticRegression(C=1, class_weight = None, solver='liblinear')

In [81]:
# Training van het model
logreg.fit(X_train, y_train)


print('De bias = theta_0: ', logreg.intercept_)
print('De coefficienten theta: ',logreg.coef_)

De bias = theta_0:  [-0.02826018]
De coefficienten theta:  [[ 0.23586972  0.08268292  0.25348959  0.36273746  0.37009201 -0.24460733
   0.7156858   0.63232973 -0.0659836  -0.30363277  1.13561566 -0.21658861
   0.59205099  0.93352236  0.32743536 -0.79838143 -0.08201114  0.06108378
  -0.4376749  -0.44216356  1.04667627  1.20969075  0.89945153  1.070317
   0.43984698  0.12875646  0.71563285  0.94709849  0.89842922  0.42192042]]


## Testen van het model op de test set

In [82]:
# Testen van het model op de test set

y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       112
           1       0.98      0.95      0.97        59

    accuracy                           0.98       171
   macro avg       0.98      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171

[[111   1]
 [  3  56]]
97.6608187134503


## Evalueren van het model

### K-fold

In [83]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = logreg, X = X_train, y = y_train, cv = 12)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 97.47 %
Standard Deviation: 2.08 %


Resultaten:
- cv = 5:
    - Accuracy: 96.72 %
    - Standard Deviation: 1.72%
- cv = 10:
    - Accuracy: 96.99 %
    - Standard Deviation: 2.69%
- cv = 12
    - Accuracy: 97.47 %
    - Standard Deviation: 2.08%
- cv = 15:
    - Accuracy: 97.23 %
    - Standard Deviation: 2.93%
- cv = 20:
    - Accuracy: 96.99%
    - Standard Deviation: 3.68%


- Varieer bij K-fold cross-validation de waarde van K. Bespreek de resultaten.

For testing the K-fold cross-validation I have used several values to see which K value is the most efficient. As you can see above, the value consits of 5, 10, 12, 15, 20. The reason why I opted for a range to 20 is due to the fact that the performance is getting worse once it has passed the 20. Moreover, the value 10 and 20 are similar with an accuracy of 96.99% Therefore, I chose the number 15 as this is the number in the middle. The results are slightly better with an accuracy of 97.47% and a standard deviation of 2.93%. To search for the most efficient k value,  I've tried once more with 12 and the results are better with an accuracy of 97.47% and a standard deviation of 2.08% which means that the range is 95.93% - 99.55% when it comes to accuracy. This is an ideal and reliable cv value when it comes to testing. 

A small note is that there is a cv value in grid search and random search but I kept it at a minimum to evaluate my model with grid search and random search to reduce the execution time. The higher the cv value, the more reliable is.

### Grid search

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
paramaters = [
             {'C' : [0.25, 0.5, 0.75, 1], 
              'solver': ['liblinear','lbfgs', 'newton-cg', 'sag','saga'], 
              'class_weight': [None, 'balanced']},
             ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 20,
                           n_jobs = -1,
                           verbose=5)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Fitting 20 folds for each of 40 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 332 tasks      | elapsed:    4.3s


Best accuracy :  0.9798684210526314
Best parameters : {'C': 0.25, 'class_weight': None, 'solver': 'liblinear'}
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       112
           1       0.98      0.93      0.96        59

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171

[[111   1]
 [  4  55]]
97.07602339181285


[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    7.4s finished


### Random search

In [85]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

parameters = [
             {'C' : uniform(0.01, 20), 
              'solver': ['liblinear','lbfgs', 'newton-cg', 'sag','saga'],
              'class_weight': [None, 'balanced']}
             ]

              

n_iter_search = 20

random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=20,n_iter=n_iter_search,n_jobs = -1,verbose=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Fitting 20 folds for each of 20 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    1.1s


Best accuracy :  0.9748684210526314
Best parameters : {'C': 11.867685804983289, 'class_weight': None, 'solver': 'saga'}
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       112
           1       0.98      0.98      0.98        59

    accuracy                           0.99       171
   macro avg       0.99      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171

[[111   1]
 [  1  58]]
98.83040935672514


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    3.3s finished
C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


- Test de bekomen modellen op de test set. Welke search techniek geniet jouw voorkeur en waarom?

Results:
    - Logistic regression on test set:
        - Accuracy: 97.66%
    - Grid search:
        - Best accuracy: 0.9798684210526314 %
        - Accuracy: 97.07%
    - Random search:
        - Best accuracy: 0.9748684210526315 %
        - Accuracy: 98.83%

The results of the logistic regression amounts 97.66% for the accuracy and the preference goes to the random search method for improving the model as the accuracy amounts 98.83% which has improved slightly.

From our confusion matrix, we can see that our model got (111+58) 169 predictions right and (1+1) 2 predictions wrong. We have 1 false positive which means that our model predicted that 1 person is malignant but it turned out not to be. This is actually good. On the other hand, our model also has 1 false negative. What this means is that our model predicted 1 person benign while it is positive. This is bad as the impact for the person will be bigger.

Moreover based on the classification report there is a preference for class 0 with 0.99% compared with class 1 with 0.98%. This model is ideal and reliable.

- Heeft het zin om de featureset uit te breiden met polynomial features? Test dit. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [90]:
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)

logreg_poly = linear_model.LogisticRegression(C=0.25, class_weight = None, solver='liblinear')

logreg_poly.fit(X_train_poly,y_train)

y_pred_poly = logreg_poly.predict(X_test_poly)

print(classification_report(y_test,y_pred_poly))

print(confusion_matrix(y_test,y_pred_poly))

print('accuracy op de test set:', accuracy_score(y_test,y_pred_poly)*100)

print('accuracy op de training set:', accuracy_score(y_train,logreg_poly.predict(X_train_poly))*100)

dimensie van X_train_poly:  (397, 496)
dimensie van X_test_poly:  (171, 496)
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       112
           1       0.93      0.92      0.92        59

    accuracy                           0.95       171
   macro avg       0.94      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171

[[108   4]
 [  5  54]]
accuracy op de test set: 94.73684210526315
accuracy op de training set: 98.99244332493703


As you can see is the accuracy on the test set 94.73% compared with the prefered method with random search. This means that there is no need for using polyonomial features. Moreover the accuracy on the training set is higher and lower on the test set. Also note the higher the degree of the polynomial, the higher the chance of overfitting.

Train een Support Vector Machine via cross-validation

- Gebruik daarvoor zowel grid search, random search en Bayes optimization om op zoek te gaan naar de beste hyperparameters: Kernel, C-waarde, Gamma waarde, graad van de polynomial kernel, class_weight. Meer info: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
- Varieer bij K-fold cross-validation de waarde van K. Bespreek de resultaten.
- Test de bekomen modellen op de test set. Welke search techniek geniet jouw voorkeur en waarom?
- Vergelijk de resultaten met deze die bekomen werden via logistic regression. Welke presteert het best in termen van accuracy, recall, precision en f1 score? Zoek een verklaring. 
- Merk je een groot verschil in berekeningstijd tussen logistic regression en Support Vector Machines? Verklaar het mogelijks verschil.
- Onderzoek of normalisatie van de features een effect heeft op de training tijd (via cross-validation). 

## Initialiseren en trainen van de SVM

In [14]:
# Trainen van SVM via cross-validation
# zonder kernel
SVMlinear = svm.SVC(kernel='linear',C=1)
SVMlinear.fit(X_train, y_train)

SVC(C=1, kernel='linear')

## Testen van het model op de test set

In [15]:
y_pred = SVMlinear.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       112
           1       0.98      0.95      0.97        59

    accuracy                           0.98       171
   macro avg       0.98      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171

[[111   1]
 [  3  56]]
97.6608187134503


## Evalueren van het model

In [41]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = SVMlinear, X = X_train, y = y_train, cv = 12)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 97.47 %
Standard Deviation: 2.08 %


Resultaten:
- cv = 5:
    - Accuracy: 95.46 %
    - Standard Deviation: 3.36%
- cv = 10:
    - Accuracy: 97.24 %
    - Standard Deviation: 2.36%
- cv = 12
    - Accuracy: 97.47 %
    - Standard Deviation: 2.08%
- cv = 15:
    - Accuracy: 96.47 %
    - Standard Deviation: 3.53%
- cv = 20:
    - Accuracy: 96.99%
    - Standard Deviation: 4.01%

- Varieer bij K-fold cross-validation de waarde van K. Bespreek de resultaten.

For testing the K-fold cross-validation I have used several values to see which K value is the most efficient. As you can see above, the value consists of 5, 10, 12, 15, 20. The reason why I opted for a range to 20 is due to the fact that the performance is getting worse once it has passed the 20. Moreover, the value 10 has an accuracy of 97.24% and 15 an accuracy of 96.47%. To search for the most efficient k value,  I've tried once more with 12 and the results are better with an accuracy of 97.47% and a standard deviation of 2.08% which means that the range is 95.93% - 99.55% when it comes to accuracy. This is an ideal and reliable cv value when it comes to testing. 

A small note is that there is a cv value in grid search and random search but I kept it at a minimum to evaluate my model with grid search and random search to reduce the execution time. The higher the cv value, the more reliable is.

### Grid search

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model = SVC()

paramaters = [ 
        {'kernel': ['linear'], 
         'C': np.linspace(0.01,20,10), 
         'degree': [2,3,4], 
         'class_weight': [None, 'balanced']},
        {'kernel': ['rbf', 'poly', 'sigmoid'], 
         'C': np.linspace(0.01,20,10), 
         'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2], 
         'degree': [2,3,4], 
         'class_weight': [None, 'balanced']},
]

grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy', 
                           cv = 20,
                           n_jobs = -1,
                           verbose =5)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )


y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


Fitting 20 folds for each of 960 candidates, totalling 19200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 2528 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 5120 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 8288 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 12032 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 16352 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 19200 out of 19200 | elapsed:   29.7s finished


Best accuracy :  0.9823684210526314
Best parameters : {'C': 2.231111111111111, 'class_weight': 'balanced', 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       112
           1       0.98      0.90      0.94        59

    accuracy                           0.96       171
   macro avg       0.97      0.94      0.95       171
weighted avg       0.96      0.96      0.96       171

[[111   1]
 [  6  53]]
95.90643274853801


### Random search

In [18]:
# Train een SVM via random search

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 
from scipy.stats import uniform


#svc trainen
model = SVC()

parameters = [
    {'kernel': ['linear'], 
     'C': uniform(0.01, 20), 
     'degree': [2,3,4], 
     'class_weight': [None, 'balanced']
    },
    {'kernel': ['rbf', 'poly', 'sigmoid'], 
     'C': uniform(0.01, 20), 
     'gamma': uniform(0.001, 0.2), 
     'degree': [2,3,4], 
     'class_weight': [None, 'balanced']}
]
 
n_iter_search = 20

random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s


Best accuracy :  0.9621518987341771
Best parameters : {'C': 0.5901916449666744, 'class_weight': None, 'degree': 2, 'gamma': 0.04173958437179073, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       112
           1       1.00      0.90      0.95        59

    accuracy                           0.96       171
   macro avg       0.97      0.95      0.96       171
weighted avg       0.97      0.96      0.96       171

[[112   0]
 [  6  53]]
96.49122807017544


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


- Test de bekomen modellen op de test set. Welke search techniek geniet jouw voorkeur en waarom?

Results:

    -SVM on test set:
        - Accuracy: 97.66%
    - Grid search:
        - Best accuracy: 0.9823684210526314 %
        - Accuracy : 95.90%
    - Random search:
        - Best accuracy: 0.9621518987341771 %
        - Accuracy : 96.49%

The results of SVM amounts 97.66% for the accuracy and the preference goes to the random search method for improving the model as the best accuracy amounts 96.49% which has improved slightly.

From our confusion matrix, we can see that our model got (112+53) 165 predictions right and (6+0) 6 predictions wrong.  We have 0 false positive which means that our model predicted that 0 person is malignant but it turned out not to be. This is actually good. On the other hand, our model also has 6 false negative. What this means is that our model predicted 6 people are benign while they are positive. This is bad as the impact for them will be bigger.

Moreover based on the classification report there is a preference for class 0 with 0.97% compared with class 1 with 0.95%. This model is ideal and reliable.

Conclusion: Based on the results we can say the the logistic regression classifier with random search the best and most efficient model is with an accuracy of 98.83%.

- Vergelijk de resultaten met deze die bekomen werden via logistic regression. Welke presteert het best in termen van accuracy, recall, precision en f1 score? Zoek een verklaring.

Results:

Logistic regression:
    - Grid search:
        - Accuracy: 97.07602339181285 %
    - Random search:
        - Accuracy: 98.83 %

SVM:
    - Grid search:
        - Accuracy: 95.90643274853801 %
    - Random search:
        - Accuracy: 96.49122807017544 %


In general, we can see that the logistic regression classifier with the random search method has the highest scores in terms of accuracy, recall, precision and f1 in the classification report. The accuracy amounts 98%, likewise all the other scores are more than 0.97%. Moreover, the model has a preference in class 0 with 0.98% and class 1 with 0.97% This means that the model is highly reliable and ideal for predicting cancer.



- Merk je een groot verschil in berekeningstijd tussen logistic regression en Support Vector Machines? Verklaar het mogelijks verschil

Yes, there is a difference in execution time between logistic regression en support vector machines. On this dataset is the SVM faster than the logistic regression. The reason is due to the fact that SVM are less complex than logistic regression. Logistic regression is also more computable intensive due to the exp function.

- Onderzoek of normalisatie van de features een effect heeft op de training tijd (via cross-validation).

Yes, feature scaling with normalisation has an impact on the execution time. If the values of the features are scaled (numbers are smaller) then this means that this is less computational.

## Opdracht 2 - MNIST
De MNIST ("Modified National Institute of Standards and Technology")is een veelgebruikte dataset voor het testen en benchmarken van klassificatie algoritmes. Het bevat tienduizenden afbeeldingen van handgeschreven getallen. 
Meer info over deze dataset is te vinden op: http://yann.lecun.com/exdb/mnist/


Bij deze opdracht worden 3 databestanden meegeleverd:
- 'train.csv' bevat gelabelde data voor **trainen** van  de classifier.
- 'test.csv' bevat gelabelde data voor het **testen** van de getrainde classifier
- 'test_Kaggle.csv' bevat niet-gelabelde data voor het evalueren van de classifier via de competitie te vinden op https://www.kaggle.com/c/digit-recognizer. Deze data is dus enkel bruikbaar bij deelname aan de Kaggle competitie. 


1) Train zowel een logistic regression classifier als een Support Vector Machine (met of zonder kernel. Ga via tuning van de hyperparameters op zoek naar de meest performante classifier. Doe dit door de accuracy te maximaliseren of de error rate te minimaliseren op de test.csv dataset. De error rate = 1- accuracy. Het trainen van Support Vector Machines (zeker deze met kernel) vragen enorm veel rekenkracht. Het is daarom verstandig om in eerste instantie te trainen op een klein deel van de training set. Het trainen via logistic regression is minder belastend voor de CPU, desalniettemin wordt aangeraden om de lbfgs solver te gebruiken. *(LogisticRegression(multi_class='multinomial', solver='lbfgs'))*

2) Verzorg de code telkens van commentaar en schrijf jouw conclusies en besluiten neer.


3) Is het nodig om hier te normaliseren? Welke normalisatie zou je gebruiken? Is de standardscaler een goede keuze?

4) Onderzoek de twee verschillende types van multiclass classification: one-vs-one (ovo) of one-vs-rest (ovr). Kijk vooral naar accuracy en berekeningstijd. Wat zijn de conclusies? 

5) Test jouw uiteindelijke classifier met een aantal zelf geschreven getallen. Wat zijn de bevindingen? Waarvan hangt classificatienauwkeurigheid af?

6) Optioneel: test op de 'test_Kaggle' dataset en laad de resultaten in het juiste formaat op naar de Kaggle website. Wat is de behaalde score? Vergelijk deze score met de score op http://yann.lecun.com/exdb/mnist/


Het visualiseren van een digit kan met *'plt.imshow(X_train[n].reshape((28, 28)),cmap = 'gray')'*


In [56]:
#Inlezen van de dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head(100)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data preprocessing

In [57]:
# Opsplitsen in features en target
X = df_train.drop(['label'], axis=1).values
y = df_train['label'].values

# Opsplitsen in training set en test set. - train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Opsplitsen in features en target
X = df_test.drop(['label'], axis=1).values
y = df_test['label'].values

# Opsplitsen in training set en test set.  - test
train_x, X_test, train_y, y_test = train_test_split(X, y, test_size=4500, random_state=0)

#print('x test:', X_test.shape)
#print('y test:', y_test.shape)

#print('x train:', X_train.shape)
#print('y train:', y_train.shape)

# Normalisatie van de features is nodig

scaler = preprocessing.StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[3 2 2 ... 3 6 9]


## Initialiseren en trainen van de logistic regression classifier

In [58]:
# Trainen van logistic regression via cross-validation

# Initialiseren van de logistic regression classifier

logreg = linear_model.LogisticRegression(C=1, class_weight = None , multi_class='multinomial', solver='lbfgs')


In [59]:
# Training van het model

logreg.fit(X_train, y_train)

print('De bias = theta_0: ', logreg.intercept_)
print('De coefficienten theta: ',logreg.coef_)

De bias = theta_0:  [-4.20834807 -4.48844875  2.56799816  3.09816911 -2.08326299  4.03747629
 -1.79047723 -2.41001208  5.12980659  0.14709897]
De coefficienten theta:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Testen van het model op de test set

In [60]:
# Testen van het model op de test set
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       436
           1       0.94      0.97      0.95       509
           2       0.84      0.84      0.84       434
           3       0.88      0.86      0.87       465
           4       0.86      0.92      0.89       412
           5       0.84      0.84      0.84       395
           6       0.91      0.94      0.92       464
           7       0.89      0.90      0.90       461
           8       0.86      0.79      0.83       428
           9       0.88      0.84      0.86       496

    accuracy                           0.88      4500
   macro avg       0.88      0.88      0.88      4500
weighted avg       0.88      0.88      0.88      4500

[[406   0   4   2   0   3  14   1   5   1]
 [  0 496   3   0   1   2   1   1   3   2]
 [ 11   5 363  16   5   2   9   6  15   2]
 [  1   3  17 399   0  22   3   6   7   7]
 [  2   2   3   0 377   3   5   0   4  16]
 [  6   5   9   9  11 333 

## Evalueren van het model

### Grid search

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# create instance logistic regression
model = LogisticRegression()
# assign parameters for grid search
paramaters = [
             {'C': [0.01, 0.01, 1, 10],  
              'solver': ['lbfgs'], 
              'class_weight': [None, 'balanced'],
              'multi_class': ['multinomial']},
             ]

# create variable grid search with the required parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 2,
                           n_jobs = -1,
                           verbose=5)

# train grid search on the training data
grid_search = grid_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

# predict on grid search with test set
y_pred = grid_search.predict(X_test)
# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:   24.5s remaining:    5.6s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   25.7s finished


Best accuracy :  0.9070666666666667
Best parameters : {'C': 0.01, 'class_weight': 'balanced', 'multi_class': 'multinomial', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       436
           1       0.94      0.99      0.96       509
           2       0.90      0.88      0.89       434
           3       0.93      0.88      0.91       465
           4       0.90      0.92      0.91       412
           5       0.88      0.86      0.87       395
           6       0.94      0.96      0.95       464
           7       0.92      0.93      0.93       461
           8       0.88      0.86      0.87       428
           9       0.90      0.88      0.89       496

    accuracy                           0.91      4500
   macro avg       0.91      0.91      0.91      4500
weighted avg       0.91      0.91      0.91      4500

[[420   0   2   0   0   3   8   0   3   0]
 [  0 503   0   0   1   1   0   1   3   0]
 [  5   1 38

C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Random search

In [62]:
# Train een SVM via random search

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression

# create instance logistic regression
model = LogisticRegression()

# assign parameters for random search
parameters = [
             {'C' : uniform(0.01, 20), 
              'solver': ['lbfgs'],
              'class_weight': [None, 'balanced'],
              'multi_class': ['multinomial']}
             ]

              
# amount ranndom search
n_iter_search = 5

# create variable random search with the required parameters
random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=2,n_iter=n_iter_search,n_jobs = -1,verbose=1)

# train random search on the training data
random_search = random_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

# predict on random search with test set
y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   15.5s finished


Best accuracy :  0.8721333333333333
Best parameters : {'C': 5.217087859902357, 'class_weight': 'balanced', 'multi_class': 'multinomial', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       436
           1       0.93      0.96      0.95       509
           2       0.82      0.83      0.82       434
           3       0.86      0.85      0.86       465
           4       0.85      0.90      0.87       412
           5       0.82      0.84      0.83       395
           6       0.92      0.94      0.93       464
           7       0.88      0.89      0.89       461
           8       0.85      0.77      0.81       428
           9       0.86      0.83      0.84       496

    accuracy                           0.87      4500
   macro avg       0.87      0.87      0.87      4500
weighted avg       0.87      0.87      0.87      4500

[[402   0   5   2   0   7  13   1   5   1]
 [  1 489   4   1   3   3   1   1   4   2]

C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results:
    - Logistic regression on test set:
        - Accuracy: 88.42%
        
    - Grid search:
        - Best accuracy:  0.8675333333333333 %
        - Accuracy:91.44%
    - Random search:
        - Best accuracy: 0.8657333333333332 %
        - Accuracy : 87.35%


The results of the logistic regression amounts 89.77% for the accuracy and the preference goes to the grid search method for improving the model as the accuracy amounts 91.44% which has improved.

In general the f1 scores from the classifcation report is in the range between 0.81 - 0.95 between the 10 classes. As the lowest is 0.81, there is room for improvement so the model can be more efficient and reliable for making predictions.



## Initialiseren en trainen van SVM

In [63]:
# Trainen van SVM via cross-validation

# Initialiseren van de SVM classifier
SVMkernel = svm.SVC(kernel='poly',C=1)
SVMkernel.fit(X_train, y_train)

SVC(C=1, kernel='poly')

## Testen van het model op de test set

In [64]:
# Training van het model

y_pred = SVMkernel.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       436
           1       0.98      0.99      0.98       509
           2       0.95      0.88      0.92       434
           3       0.97      0.91      0.94       465
           4       0.86      0.96      0.91       412
           5       0.95      0.90      0.93       395
           6       0.97      0.93      0.95       464
           7       0.97      0.90      0.93       461
           8       0.77      0.96      0.85       428
           9       0.93      0.91      0.92       496

    accuracy                           0.93      4500
   macro avg       0.93      0.93      0.93      4500
weighted avg       0.93      0.93      0.93      4500

[[416   0   1   0   0   0   9   0  10   0]
 [  1 503   0   0   1   0   0   0   4   0]
 [  4   0 383   3   5   0   1   1  37   0]
 [  0   2   6 421   1   4   0   1  27   3]
 [  1   2   3   1 394   2   1   0   2   6]
 [  1   0   2   4   4 357 

## Evalueren van het model

### Grid search

In [65]:


from sklearn.model_selection import GridSearchCV
#from sklearn.learning_curve import learning_curve
from sklearn.svm import SVC
#from sklearn import cross_validation
from sklearn.datasets import load_digits
from sklearn.model_selection import validation_curve


# create instance SVC
model = SVC()

# assign parameters for random search
paramaters = [ 
        {'kernel': ['rbf', 'poly', 'sigmoid'], 
         'C': [0.01, 1, 10], 
         'gamma': [0.001, 0.01, 0.1], 
         'class_weight': [None, 'balanced'], 
         'decision_function_shape': ['ovo', 'ovr']},
        ]

# create variable grid search with the required parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 2, 
                           n_jobs = -1,
                           verbose = 5)

# train grid search on the training data
grid_search = grid_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

# predict on grid search with test set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

Fitting 2 folds for each of 108 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 66.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 146.6min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 183.3min finished


Best accuracy :  0.9507333333333334
Best parameters : {'C': 0.01, 'class_weight': None, 'decision_function_shape': 'ovo', 'gamma': 0.1, 'kernel': 'poly'}
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       436
           1       0.99      1.00      0.99       509
           2       0.95      0.96      0.96       434
           3       0.95      0.97      0.96       465
           4       0.94      0.97      0.95       412
           5       0.97      0.94      0.96       395
           6       0.98      0.96      0.97       464
           7       0.96      0.96      0.96       461
           8       0.94      0.95      0.95       428
           9       0.96      0.93      0.95       496

    accuracy                           0.96      4500
   macro avg       0.96      0.96      0.96      4500
weighted avg       0.96      0.96      0.96      4500

[[425   0   2   0   0   1   7   0   1   0]
 [  1 507   0   0   0   0   0   0   1   0]
 

### Random search

In [66]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 
from scipy.stats import uniform


# create instance SVC
model = SVC()

# assign parameters for random search
parameters = [
    {'kernel': ['rbf', 'poly', 'sigmoid'], 
     'C': uniform(0.01, 20), 
     'gamma': uniform(0.001, 0.2), 
     'class_weight': [None, 'balanced'], 
     'decision_function_shape': ['ovo', 'ovr']},
    ]


# amount random search
n_iter_search = 5

# create variable random search with the required parameters
random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=2,n_iter=n_iter_search,n_jobs = -1,verbose=1)

# train random search on the training data
random_search = random_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

# predict on random search with test set
y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.8min finished


Best accuracy :  0.9507333333333334
Best parameters : {'C': 18.453429347029257, 'class_weight': None, 'decision_function_shape': 'ovr', 'gamma': 0.17460634715043422, 'kernel': 'poly'}
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       436
           1       0.99      1.00      0.99       509
           2       0.95      0.96      0.96       434
           3       0.95      0.97      0.96       465
           4       0.94      0.97      0.95       412
           5       0.97      0.94      0.96       395
           6       0.98      0.96      0.97       464
           7       0.96      0.96      0.96       461
           8       0.94      0.95      0.95       428
           9       0.96      0.93      0.95       496

    accuracy                           0.96      4500
   macro avg       0.96      0.96      0.96      4500
weighted avg       0.96      0.96      0.96      4500

[[425   0   2   0   0   1   7   0   1   0]
 [  1 507   0 

Results:

    -SVM on test set:
        - Accuracy: 92.86%
    - Grid search:
        - Best accuracy: 0.9507 %
        - Accuracy: 96.11%
    - Random search:
        - Best accuracy: 0.9507 %
        - Accuracy 96.11%

The results of the logistic regression amounts 92.86% for the accuracy and the preference goes to the grid and random search method for improving the model as the accuracy are both similar with 96.11%. The model has improved.

In general the f1 scores from the classifcation report is in the range between 0.95 - 0.99 between the 10 classes. As the lowest This is an ideal model for making predictions.

Conclusion: Based on the results we can say the the SVM classifier with grid search or random search method gets the best model with an accuracy of 96.11%

3) Is het nodig om hier te normaliseren? Welke normalisatie zou je gebruiken? Is de standardscaler een goede keuze?

Feature scaling is not required but always recommended as the feature values will be scaled and that it will speed up the execution time. Standardisation is a technique that pretty much works all the time compared with the other techniques. Therefore, it is recommended to use standardscaler as it always will do some relevant feature scaling and improve the training process.

4) Onderzoek de twee verschillende types van multiclass classification: one-vs-one (ovo) of one-vs-rest (ovr). Kijk vooral naar accuracy en berekeningstijd. Wat zijn de conclusies?

Between the 2 different types of multiclass classification is the one-vs-rest (over) the assigned parameter for both grid and random search. In this case, the ovr is the best parameter which means that it certainly improves the accuracy rate and execution time.

5) Test jouw uiteindelijke classifier met een aantal zelf geschreven getallen. Wat zijn de bevindingen? Waarvan hangt classificatienauwkeurigheid af?

## Opdracht 3 - 50K a year

Train een classifier die zo accuraat mogelijk kan voorspellen of iemand al dan niet dan 50000 dollar per jaar verdient. Gebruik hiervoor de dataset '50K_a_year.csv'.

Train zowel een logistic regression classifier als een Support Vector Machine (met of zonder kernel. Ga via tuning van de hyperparameters op zoek naar de meest performante classifier. Bespreek en argumentaar welke classifier jouw voorkeur geniet. 


In [92]:
dataset = pd.read_csv('50K_a_year.csv')
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## Data exploration

In [93]:
# inconsistenties opsporen
dataset.isnull().sum()
# dataset.isna().any().any()
# neen, er zijn geen ontbrekende waarden

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [94]:
dataset.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [95]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


## Data preprocessing

In [96]:
# remove columns

dataset.drop(['capital-gain'],axis=1, inplace=True)
dataset.drop(['capital-loss'],axis=1, inplace=True)
dataset.drop(['education'], axis=1, inplace=True)

# one hot encoding
dataset = pd.concat([dataset,pd.get_dummies(dataset['workclass'], prefix='workclass')],axis=1)
dataset.drop(['workclass'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['marital-status'], prefix='marital-status')],axis=1)
dataset.drop(['marital-status'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['occupation'], prefix='occupation')],axis=1)
dataset.drop(['occupation'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['relationship'], prefix='relationship')],axis=1)
dataset.drop(['relationship'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['race'], prefix='race')],axis=1)
dataset.drop(['race'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['native-country'], prefix='country')],axis=1)
dataset.drop(['native-country'],axis=1, inplace=True)

dataset = pd.concat([dataset,pd.get_dummies(dataset['sex'], prefix='sex')],axis=1)
dataset.drop(['sex'],axis=1, inplace=True)

dataset['class'] = dataset['class'].map({' >50K': 1, ' <=50K': 0})

#print(dataset.columns.tolist())
# workclass_ ?
dataset.drop(['workclass_ ?'], axis=1, inplace=True)
# occupation_ ?
dataset.drop(['occupation_ ?'], axis=1, inplace=True)
# country_ ?
dataset.drop(['country_ ?'], axis=1, inplace=True)

dataset

,age,fnlwgt,education-num,hours-per-week,class,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,...,country_ Scotland,country_ South,country_ Taiwan,country_ Thailand,country_ Trinadad&Tobago,country_ United-States,country_ Vietnam,country_ Yugoslavia,sex_ Female,sex_ Male
0,25,226802,7,40,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1,38,89814,9,50,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
2,28,336951,12,40,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,44,160323,10,40,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
4,18,103497,10,30,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,257302,12,38,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
48838,40,154374,9,40,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
48839,58,151910,9,40,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
48840,22,201490,9,20,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [97]:
from sklearn.preprocessing import StandardScaler
# Opsplitsen in features en targets
X = dataset.drop(['class'], axis=1).values
y = dataset['class'].values

# Opsplitsen in training set en test set. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Shape:',X_train.shape)

# Normalisatie van de features
scaler = StandardScaler()

X_train[:,:4] = scaler.fit_transform(X_train[:,:4])
X_test[:,:4] = scaler.transform(X_test[:,:4])


Shape: (39073, 87)


## Initialiseren en trainen van de logistic regression classifier

In [10]:
# Trainen van logistic regression via cross-validation

# Initialiseren van de logistic regression classifier

logreg = linear_model.LogisticRegression(C=1, class_weight = None, solver='lbfgs')

In [11]:
# Training van het model
logreg.fit(X_train, y_train)


print('De bias = theta_0: ', logreg.intercept_)
print('De coefficienten theta: ',logreg.coef_)

De bias = theta_0:  [-2.39415616]
De coefficienten theta:  [[ 0.23748732  0.11492111  0.99590255  0.32734201  0.95113027  0.22618169
  -0.14353214  0.33337906  0.7755611  -0.06777578 -0.06044241 -0.48151403
  -0.55524405  1.11996837  1.31427119 -0.58561712 -1.26957863 -0.73557227
  -0.40822588  0.24117175  0.22876158  0.2595456   1.15127216 -0.56579132
  -0.53230983 -0.12558763 -0.68826009 -1.45725952  1.00606893  0.62507425
   0.57097198  0.85416471  0.10869734 -0.15621473  0.17150855 -0.79499975
  -1.10945719 -0.08544804  0.85461277 -0.62196867  0.05624469 -0.34883068
  -0.13019417 -0.07524956  0.52983656  0.67120149 -0.42351274 -1.56937658
   0.11657618 -0.89515304 -0.36062697 -0.7289751   0.66776965  0.55053382
   0.15911992  0.00686893 -0.30890796 -0.12018055 -0.00302039 -0.00745274
  -0.68283427  0.25321276  0.04006241  0.17492427  0.5083663   0.73958392
   0.34617966  0.20253042 -0.71280657 -0.49440196 -0.5740015  -0.65268415
  -0.97023628  0.38759792 -0.04798908  0.38965138 -0.

C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Testen van het model op de test set

In [12]:
# Testen van het model op de test set

y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      7420
           1       0.68      0.53      0.60      2349

    accuracy                           0.83      9769
   macro avg       0.77      0.73      0.74      9769
weighted avg       0.82      0.83      0.82      9769

[[6830  590]
 [1103 1246]]
82.66966936226841


## Evalueren van het model

### Grid search

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# create instance logistic regression
model = LogisticRegression()
# assign parameters for grid search
paramaters = [
             {'C' : [0.01, 1, 10], 
              'solver': ['liblinear','lbfgs', 'newton-cg', 'sag','saga'], 
              'class_weight': [None, 'balanced'],
              'multi_class': ['multinomial', 'ovr'],
             }
             ]

# create variable grid search with the required parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 2,
                           n_jobs = -1,
                           verbose=5)

# train grid search on the training data
grid_search = grid_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

# predict on grid search with test set
y_pred = grid_search.predict(X_test)
# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.3min finished


Best accuracy :  0.833439930616122
Best parameters : {'C': 1, 'class_weight': None, 'multi_class': 'ovr', 'solver': 'saga'}
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      7420
           1       0.68      0.53      0.59      2349

    accuracy                           0.83      9769
   macro avg       0.77      0.72      0.74      9769
weighted avg       0.82      0.83      0.82      9769

[[6833  587]
 [1110 1239]]
82.62872351315386


C:\Users\Dominic Ho\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


### Random search

In [14]:
# Train een SVM via random search

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression

# create instance logistic regression
model = LogisticRegression()

# assign parameters for random search
parameters = [
             {'C' : uniform(0.01, 20), 
              'solver': ['liblinear','lbfgs', 'newton-cg', 'sag','saga'],
              'class_weight': [None, 'balanced'],
              'multi_class': ['multinomial', 'ovr']},
             ]

              
# amount ranndom search
n_iter_search = 5

# create variable random search with the required parameters
random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

# train random search on the training data
random_search = random_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

# predict on random search with test set
y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   29.0s finished


Best accuracy :  0.8327743844774893
Best parameters : {'C': 8.568324096769386, 'class_weight': None, 'multi_class': 'ovr', 'solver': 'liblinear'}
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      7420
           1       0.68      0.53      0.59      2349

    accuracy                           0.83      9769
   macro avg       0.77      0.72      0.74      9769
weighted avg       0.82      0.83      0.82      9769

[[6834  586]
 [1108 1241]]
82.65943289998975


The results of the logistic regression amounts 82.66% for the accuracy and the preference goes to the grid search method for improving the model as the best accuracy amounts 0.8334% which has improved slightly with a cv value of 2. 

From our confusion matrix, we can see that our model got (6833+1239) 8072 predictions right and (1110+587) 1697 predictions wrong. We have 587 false positive which means that our model predicted that 587 person were earning more than 50k but it turned out not to be. On the other hand, our model also has 1110 false negative. What this means is that our model predicted 1110 people earning less than 50k while they are positive. 

Moreover based on the classification report there is a preference for class 0 with 0.89% compared with class 1 with 0.59%. This model is ok but can certainly be improved to be in the range of 0.90-1.00% which makes the model ideal.

## Initialiseren en trainen van SVM

In [17]:
# Trainen van SVM via cross-validation
SVMlinear = svm.SVC(kernel='linear',C=1)
SVMlinear.fit(X_train, y_train)

SVC(C=1, kernel='linear')

## Testen van het model op de test set

In [18]:
# Testen van het model op de test set

y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      7420
           1       0.68      0.53      0.60      2349

    accuracy                           0.83      9769
   macro avg       0.77      0.73      0.74      9769
weighted avg       0.82      0.83      0.82      9769

[[6830  590]
 [1103 1246]]
82.66966936226841


## Evalueren van het model

### Grid search

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# create instance SVC
model = SVC()

# assign parameters for random search
paramaters = [ 
        {'kernel': ['linear'], 
         'C': [0.01, 1, 10],
         'degree': [2,3,4],
         'class_weight': [None, 'balanced'],
         'decision_function_shape': ['ovo', 'ovr'],
        },
        {'kernel': ['rbf', 'poly', 'sigmoid'], 
         'C': [0.01, 1, 10], 
         'gamma': [0.01, 0.1, 0.2],
         'degree': [2,3,4],
         'class_weight': [None, 'balanced'], 
         'decision_function_shape': ['ovo', 'ovr'],
        },
        ]

# create variable grid search with the required parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 2,
                           n_jobs = -1,
                           verbose =5)

# train grid search on the training data
grid_search = grid_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

# predict on grid search with test set
y_pred = grid_search.predict(X_test)
# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


Fitting 2 folds for each of 360 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 59.7min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 131.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 197.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 308.7min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 363.3min finished


Best accuracy :  0.8353594310352641
Best parameters : {'C': 1, 'class_weight': None, 'decision_function_shape': 'ovo', 'degree': 2, 'gamma': 0.2, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      7420
           1       0.70      0.53      0.60      2349

    accuracy                           0.83      9769
   macro avg       0.78      0.73      0.75      9769
weighted avg       0.82      0.83      0.82      9769

[[6877  543]
 [1098 1251]]
83.20196540075749


### Random search

In [20]:
# Train een SVM via random search

from sklearn.model_selection import RandomizedSearchCV
# distributions
from scipy.stats import randint 
from scipy.stats import uniform


# create instance SVC
model = SVC()

# assign parameters for random search
parameters = [
    {'kernel': ['linear'], 
     'C': uniform(0.01, 20),
     'degree': [2,3,4],
     'class_weight': [None, 'balanced'], 
     'decision_function_shape': ['ovo', 'ovr'],
    },
    {'kernel': ['rbf', 'poly', 'sigmoid'], 
     'C': uniform(0.01, 20),
     'degree': [2,3,4],
     'gamma': uniform(0.001, 0.2), 
     'class_weight': [None, 'balanced'], 
     'decision_function_shape': ['ovo', 'ovr'],
    }
    ]

# amount ranndom search
n_iter_search = 5

# create variable random search with the required parameters
random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

# train random search on the training data
random_search = random_search.fit(X_train, y_train)

# assign the best score and best parameters to variables
best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

# predict on random search with test set
y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

# confusion matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 40.9min finished


Best accuracy :  0.8212575325780571
Best parameters : {'C': 17.15682103644547, 'class_weight': None, 'decision_function_shape': 'ovr', 'degree': 2, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      7420
           1       0.63      0.54      0.58      2349

    accuracy                           0.81      9769
   macro avg       0.75      0.72      0.73      9769
weighted avg       0.81      0.81      0.81      9769

[[6687  733]
 [1076 1273]]
81.48223973794656


The results of the SVM amounts 82.66% for the accuracy and the preference goes to the grid search method for improving the model as the best accuracy amounts 83.20% which has improved slightly.

From our confusion matrix, we can see that our model got (6877+1251) 8128 predictions right and (1098+543) 1641 predictions wrong. We have 543 false positive which means that our model predicted that 543 person were earning more than 50k but it turned out not to be. On the other hand, our model also has 1098 false negative. What this means is that our model predicted 1098 people earning less than 50k while they are positive. 

Moreover based on the classification report there is a preference for class 0 with 0.89% compared with class 1 with 0.60%. This model is ok but can certainly be improved to be in the range of 0.90-1.00% which makes the model ideal.

Conclusion: Based on the results, the SVM classifier with the grid search method is the best and most efficient model with an accuracy of 83.20%